In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("datacon-22/refined_data/train_data.csv")
df.describe()

,rating_score
count,50000.000000
mean,4.088940
std,1.346798
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [3]:
df['rating_score'].unique()

array([4, 5, 3, 2, 1], dtype=int64)